In [107]:
import sys
sys.executable

'/home/goonieb/gitrep/bandstour/venv/bin/python'

In [108]:
import os
import json
import pymongo
import musicbrainzngs
import arrow
import re
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict, OrderedDict

from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [109]:
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
# setup mongo
client = MongoClient()
db = client["bandstour"]
venues =  db["selectedVenues"]
artists =  db["selectedArtists"]

def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.
    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).
    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
"(or 'y' or 'n').\n")

def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    s = re.sub('\s+', ' ', s)
    s = s.replace(' ', '-')

    s = re.sub('\W+', ' ', s)

    s = s.strip()

    s = s.lstrip()
    s = re.sub('\W+', '_', s)
    return s

In [97]:
#WE WANT TO LOOP OVER VENUES TO ELIMINATE RESIDUAL TEST DATAS (BINTOWN) AND  FIX THIS, AND DEDUPLICATE VENUES
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose
for venueA in venues.find({},no_cursor_timeout=True) :
    
    #FIRST THE ONES SURELY WRONG:
    
    
    if  (venueA["name"] in ["TEST NEW VENUE","TEST VENUE","Test ","Test 8","Test Facility","Test German Venue","Test Only","Test Vanue","Test Venu","Test Venue","Test Venue 1","Test Venue DJ 11","Test Venue Marquee","Test de Marco","Test event","Test venue","Test venue 62","Test_Venue_100708","Testing a Venue","test again vm","test event","test ire venue","test test test","test test test 1","test test test 2","test venue ","test venue 9","test2","test3","testdate","teste"]  ):
        ####extracted via salles_a_trier.ods
        ###and (venueA["latitude"]!= "37.7909889" and venueA["longitude"]!= "-122.3990707"))
            print "AAAA  =>", venueA["_id"]
            print slugify(venueA["_id"])
            print venueA
            #check if this test venue exists and it is a typo
            print " TO DELETE, NOW SHOWING ARTISTS TO ALTER"

            for artist in artists.find():
                             
                                 for gig   in artist["gigs"]:
                                    if (gig["venue"]["name"]==venueA["_id"] and round(gig["venue"]["latitude"],6)== round(venueA["latitude"],6) and round(gig["venue"]["longitude"],6)== round(venueA["longitude"],6) ):
                                        #[{u'venue': {u'city': u'Montreal', u'name': u'Medley', u'country': u'Canada', u'region': u'QC', u'longitude': -73.554392, u'place': u'Medley', u'latitude': 45.512288}, u'datetime': datetime.datetime(2002, 4, 14, 20, 30, 16)}, {u'venue': {u'city': u'Boston', u'name': u'Test Venue', u'country': u'United States', u'region': u'MA', u'longitude': -71.0602778, u'place': u'Test Venue', u'latitude': 42.3583333}, u'datetime': datetime.datetime(2010, 1, 2, 19, 0)}]
                                        
                                        
                                        print len(artist["gigs"])
                                        
                                        print "_______________"
                                        print gig
                                        print "_-_-_-_-_-_-_-_-"
                                        print artist["_id"] , gig["venue"],"****************************************************"    
                                        print "______---------------__________"
                                        print "REMOVING"
                                        artist["gigs"].remove(gig)
                                        print len(artist["gigs"])
                                        artists.update({"_id" :artist["_id"]}, artist, upsert=True, multi=False)
            print "DONE WITH THIS VENUE1"           
            
            

        

AAAA  => test venue 
test_venue
{u'count': 5, u'city': u'Moscow', u'name': u'test venue ', u'country': u'Russian Federation', u'region': u'48', u'longitude': 37.6155556, u'latitude': 55.7522222, u'_id': u'test venue ', u'id': u'test venue '}
 TO DELETE, NOW SHOWING ARTISTS TO ALTER
DONE WITH THIS VENUE1
AAAA  => Test Venue
Test_Venue
{u'count': 139, u'city': u'Atka', u'name': u'Test Venue', u'country': u'United States', u'region': u'AK', u'longitude': -174.2005556, u'latitude': 52.1961111, u'_id': u'Test Venue', u'id': u'Test Venue'}
 TO DELETE, NOW SHOWING ARTISTS TO ALTER
DONE WITH THIS VENUE1


In [125]:
     #THEN THE PROBABLY RIGHT ONES:       
            
for venueA in venues.find({},no_cursor_timeout=True) :
    
    #we check for test venues
    if  (venueA["name"] in ["Test","Test (new)","Test 17:01","Test Again","Test Club","Test Past Event","Test Vannue","Test Venues","Test Venz","Test Xavier","Test2","Test3","TestVenue","test ven","test venue 1","testing"] ) :
        ####extracted via salles_a_trier.ods
            print "AAAA  =>", venueA["_id"]
            print slugify(venueA["_id"])
            print venueA
            #check if this test venue exists and it is a typo
            counter=0
            for artist in artists.find():
                            for gig   in artist["gigs"]:
                                if ( round(gig["venue"]["latitude"],6)== round(venueA["latitude"],6) and round(gig["venue"]["longitude"],6)== round(venueA["longitude"],6) ):
                                    counter+=1
                                    #print artist["_id"] , gig["venue"],"****************************************************"
                                    #print counter
                                    if counter==20:
                                        break
                            if counter==20:
                                print venueA["name"]
                                print venueA,"could be replaced       with      ",gig["venue"]
                                print gig["venue"]["name"]
                                print "NOW SHOWING ARTISTS TO ALTER"
                                venueTMP=gig["venue"]
                                break
            for artist in artists.find():
                                 
                                 modded=0
                                 for gigB   in artist["gigs"]:
                                    if (gigB["venue"]["name"]==venueA["name"] and round(gigB["venue"]["latitude"],6)== round(venueA["latitude"],6) and round(gigB["venue"]["longitude"],6)== round(venueA["longitude"],6) ):
                                        print "GIG",gigB
                                        modded=1
                                        gigTMP=gigB
                                        artist["gigs"].remove(gigB)
                                        if "venueTMP" in locals():
                                            gigTMP["venue"]=venueTMP
                                            print "GIGTMP",gigTMP
                                            artist["gigs"].append(gigTMP)
                                        #print artist["_id"] , artist["gigs"],"****************************************************"    
                                        
                                    else:
                                        pass
                                 if modded==1:
                                    pass
                                    print "MODDED"
                                    #print artist
                                    artists.update({"_id" :artist["_id"]}, artist, upsert=True, multi=False) 
            if "venueTMP" in locals():
                del  venueTMP                       
            print "DONE WITH THIS VENUE2"           
            

AAAA  => Test Venz
Test_Venz
{u'count': 5, u'city': u'Boston', u'name': u'Test Venz', u'country': u'United States', u'region': u'MA', u'longitude': -71.0602778, u'latitude': 42.3583333, u'_id': u'Test Venz', u'id': u'Test Venz'}
Test Venz
{u'count': 5, u'city': u'Boston', u'name': u'Test Venz', u'country': u'United States', u'region': u'MA', u'longitude': -71.0602778, u'latitude': 42.3583333, u'_id': u'Test Venz', u'id': u'Test Venz'} could be replaced       with       {u'city': u'Allston', u'name': u'Great Scott', u'country': u'United States', u'region': u'MA', u'longitude': -71.0602778, u'place': u'Great Scott', u'latitude': 42.3583333}
Great Scott
NOW SHOWING ARTISTS TO ALTER
DONE WITH THIS VENUE2
AAAA  => Test
Test
{u'count': 86, u'city': u'Blukbuk', u'name': u'Test', u'country': u'Indonesia', u'region': None, u'longitude': 106.416641, u'latitude': -6.10477, u'_id': u'Test', u'id': u'Test'}
GIG {u'venue': {u'city': u'Blukbuk', u'name': u'Test', u'country': u'Indonesia', u'region': 

/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:49: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


GIG {u'venue': {u'city': u'Blukbuk', u'name': u'Test', u'country': u'Indonesia', u'region': None, u'longitude': 106.416641, u'place': u'Test', u'latitude': -6.10477}, u'datetime': datetime.datetime(2013, 11, 12, 19, 0)}
MODDED
GIG {u'venue': {u'city': u'Blukbuk', u'name': u'Test', u'country': u'Indonesia', u'region': None, u'longitude': 106.416641, u'place': u'Test', u'latitude': -6.10477}, u'datetime': datetime.datetime(2014, 1, 24, 19, 0)}
MODDED
GIG {u'venue': {u'city': u'Blukbuk', u'name': u'Test', u'country': u'Indonesia', u'region': None, u'longitude': 106.416641, u'place': u'Test', u'latitude': -6.10477}, u'datetime': datetime.datetime(2011, 2, 17, 19, 0)}
MODDED
GIG {u'venue': {u'city': u'Blukbuk', u'name': u'Test', u'country': u'Indonesia', u'region': None, u'longitude': 106.416641, u'place': u'Test', u'latitude': -6.10477}, u'datetime': datetime.datetime(2014, 10, 9, 15, 45)}
GIG {u'venue': {u'city': u'Blukbuk', u'name': u'Test', u'country': u'Indonesia', u'region': None, u'l

In [151]:

for venueA in venues.find({},no_cursor_timeout=True) :
    if  "count" not in venueA:
        
                print venueA, "NO COUNT!!!!!!!!!"
    elif  isinstance(venueA["count"], (int, long)):
        

In [165]:
###....AND DEDUPLICATE VENUES
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose
print "DEDUPLICATOR V.60"
for venueA in venues.find({},no_cursor_timeout=True) :
            
    
    #we check for test venues
    
            for venueB in venues.find({},no_cursor_timeout=True) :
                if venueA["_id"]==venueB["_id"]:
                    pass
                    #print venueA["_id"],'SAME'
                elif (fuzz.ratio(\
                                        slugify(venueA["_id"]).lower()\
                                        , slugify(venueB["_id"]).lower()\
                                       ) > 75 and venueA["count"] >= venueB["count"]) :
                    
           
                    
                    if ( round(venueA["latitude"],1)!= round(venueB["latitude"],1) and round(venueA["longitude"],1)!= round(venueB["longitude"],1) ):
                        #print "TOO FAR, SKIPPED"
                        pass
                    else:
                        print "_______________________________"
                        print "AAAA  =>", venueA["_id"]
                        #print venueA
                        #print "AAAA slug  =>",slugify(venueA["_id"]).lower()
                        #print "AAAA lat/long =>",venueA["latitude"],venueA["longitude"]
                        print "appxAAA ltlng=>",round(venueA["latitude"],3),round(venueA["longitude"],3)
                        print "AAAA gigs =>", venueA["count"]
                        print "BBBB  =>",venueB["_id"]
                        #print venueB
                        #print "BBBB slug  =>",slugify(venueB["_id"]).lower()
                        #print "BBBB lat/long =>",venueB["latitude"],venueB["longitude"]
                        print "appxBBB ltlng=>",round(venueB["latitude"],3),round(venueB["longitude"],3)
                        print "BBBB gigs =>", venueB["count"]
                        Answer_art= query_yes_no("MERGE "+venueB["_id"]+ " WITH "+ venueA["_id"]+" ?" , default="no")
                        print Answer_art
                        if Answer_art == True :
                         
                         for artist in artists.find():
                            counterREMPLC_ART=0
                            for gig   in artist["gigs"]:
                                if (gig["venue"]["name"]==venueB["name"] and round(gig["venue"]["latitude"],1)== round(venueB["latitude"],1) and round(gig["venue"]["longitude"],1)== round(venueB["longitude"],1) ):
                                    print artist["_id"] , gig["venue"],"****************************************************"
                                    gigTMPdedup=gig
                                    counterREMPLC_ART+=1
                                    #print "*-------"
                                    #print gig
                                    #print "----------"
                                    artist["gigs"].remove(gig)
                                    gigTMPdedup["venue"]=venueA
                                    
                                    #gigTMPdedup["venue"].pop("count",None )
                                    #gigTMPdedup["venue"].pop("id",None )
                                    artist["gigs"].append(gigTMPdedup)
                                    #print gigTMPdedup
                                    #print venueA
                                    #print "venuecountCHECK",venueA["count"],venueB["count"]
                                    #venueA["count"] += 1
                                    #venueB["count"] -= 1
                                    #print "venuecountCHECK",venueA["count"],venueB["count"]
                                    #[4 if x==1 else x for x in a]
                            
                            if counterREMPLC_ART > 0 :
                                print counterREMPLC_ART
                                artists.update({"_id" :artist["_id"]}, artist, upsert=True, multi=False)
                                
                         venueA["count"] += venueB["count"]  
                         venues.update({"_id" :venueA["_id"]}, venueA, upsert=True, multi=False)
                         #       if venueB["count"] == 0:
                         venues.remove({"_id" :venueB["_id"]})
                         #           print venueB["name"] ,"removed"
                         #       else:
                         #           print venueB["name"] ,"updated"
                         #                                           
                         #           venues.update({"_id" :venueB["_id"]}, venueB, upsert=True, multi=False)
                            
                    

DEDUPLICATOR V.60
_______________________________
AAAA  => The Cupping Room
appxAAA ltlng=> 40.723 -74.003
AAAA gigs => 19
BBBB  => The Counting Room
appxBBB ltlng=> 40.721 -73.957
BBBB gigs => 5
MERGE The Counting Room WITH The Cupping Room ? [y/N] n
False
_______________________________
AAAA  => The Cupping Room
appxAAA ltlng=> 40.723 -74.003
AAAA gigs => 19
BBBB  => THE COUNTING ROOM
appxBBB ltlng=> 40.721 -73.957
BBBB gigs => 6
MERGE THE COUNTING ROOM WITH The Cupping Room ? [y/N] n
False
_______________________________
AAAA  => Lincoln Lake Baptist Youth Camp 
appxAAA ltlng=> 43.252 -85.302
AAAA gigs => 10
BBBB  => Lake Effect @ Lincoln Lake Baptist Youth Camp
appxBBB ltlng=> 43.252 -85.302
BBBB gigs => 9
MERGE Lake Effect @ Lincoln Lake Baptist Youth Camp WITH Lincoln Lake Baptist Youth Camp  ? [y/N] y
True
The Birdsongs {u'city': u'Gowen', u'name': u'Lake Effect @ Lincoln Lake Baptist Youth Camp', u'country': u'United States', u'region': u'MI', u'longitude': -85.3019444, u'place

/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:65: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:68: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:70: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


_______________________________
AAAA  => Lincoln Lake Baptist Youth Camp 
appxAAA ltlng=> 43.252 -85.302
AAAA gigs => 19
BBBB  => Lincoln Lake Baptist Youth Camp
appxBBB ltlng=> 43.252 -85.302
BBBB gigs => 16
MERGE Lincoln Lake Baptist Youth Camp WITH Lincoln Lake Baptist Youth Camp  ? [y/N] y
True
The Birdsongs {u'city': u'Gowen', u'name': u'Lincoln Lake Baptist Youth Camp', u'country': u'United States', u'region': u'MI', u'longitude': -85.3019444, u'place': u'Lincoln Lake Baptist Youth Camp', u'latitude': 43.2516667} ****************************************************
The Birdsongs {u'city': u'Gowen', u'name': u'Lincoln Lake Baptist Youth Camp', u'country': u'United States', u'region': u'MI', u'longitude': -85.3019444, u'place': u'Lincoln Lake Baptist Youth Camp', u'latitude': 43.2516667} ****************************************************
The Birdsongs {u'city': u'Gowen', u'name': u'Lincoln Lake Baptist Youth Camp', u'country': u'United States', u'region': u'MI', u'longitude': -85

_______________________________
AAAA  => MS Silja Symphony
appxAAA ltlng=> 59.333 18.05
AAAA gigs => 14
BBBB  => Silja Symphony
appxBBB ltlng=> 59.333 18.05
BBBB gigs => 6
MERGE Silja Symphony WITH MS Silja Symphony ? [y/N] y
True
Eric Saade {u'city': u'Stockholm', u'name': u'Silja Symphony', u'country': u'Sweden', u'region': u'26', u'longitude': 18.05, u'place': u'Silja Symphony', u'latitude': 59.3333333} ****************************************************
1
_______________________________
AAAA  => DC10/Circoloco
appxAAA ltlng=> 38.909 1.433
AAAA gigs => 7
BBBB  => DC 10 / Circoloco
appxBBB ltlng=> 38.9 1.433
BBBB gigs => 5
MERGE DC 10 / Circoloco WITH DC10/Circoloco ? [y/N] y
True
Ellen Allien {u'city': u'Ibiza', u'name': u'DC 10 / Circoloco', u'country': u'Spain', u'region': u'07', u'longitude': 1.4333333, u'place': u'DC 10 / Circoloco', u'latitude': 38.9} ****************************************************
Ellen Allien {u'city': u'Ibiza', u'name': u'DC 10 / Circoloco', u'country'

_______________________________
AAAA  => Monday-Continental Club
appxAAA ltlng=> 30.251 -97.749
AAAA gigs => 14
BBBB  => SUNDAYS @ The Continental Club
appxBBB ltlng=> 30.267 -97.743
BBBB gigs => 5
MERGE SUNDAYS @ The Continental Club WITH Monday-Continental Club ? [y/N] y
True
Amanda Cevallos {u'city': u'Austin', u'name': u'SUNDAYS @ The Continental Club', u'country': u'United States', u'region': u'TX', u'longitude': -97.7427778, u'place': u'SUNDAYS @ The Continental Club', u'latitude': 30.2669444} ****************************************************
Amanda Cevallos {u'city': u'Austin', u'name': u'SUNDAYS @ The Continental Club', u'country': u'United States', u'region': u'TX', u'longitude': -97.7427778, u'place': u'SUNDAYS @ The Continental Club', u'latitude': 30.2669444} ****************************************************
Amanda Cevallos {u'city': u'Austin', u'name': u'SUNDAYS @ The Continental Club', u'country': u'United States', u'region': u'TX', u'longitude': -97.7427778, u'place

...

MERGE The Scotland Yard Pub WITH The Scotland Yard ? [y/N] y
True
Alyson's Anthem {u'city': u'Los Angeles', u'name': u'The Scotland Yard Pub', u'country': u'United States', u'region': u'CA', u'longitude': -118.60745, u'place': u'The Scotland Yard Pub', u'latitude': 34.201065} ****************************************************
1
Nigel Walsh {u'city': u'Los Angeles', u'name': u'The Scotland Yard Pub', u'country': u'United States', u'region': u'CA', u'longitude': -118.60745, u'place': u'The Scotland Yard Pub', u'latitude': 34.201065} ****************************************************
Nigel Walsh {u'city': u'Los Angeles', u'name': u'The Scotland Yard Pub', u'country': u'United States', u'region': u'CA', u'longitude': -118.60745, u'place': u'The Scotland Yard Pub', u'latitude': 34.201065} ****************************************************
Nigel Walsh {u'city': u'Los Angeles', u'name': u'The Scotland Yard Pub', u'country': u'United States', u'region': u'CA', u'longitude': -118.60745, u

Brandon Holder {u'city': u'Nashville', u'name': u'Barlines at the Omni Nashville Hotel', u'country': u'United States', u'region': u'TN', u'longitude': -86.7844444, u'place': u'Barlines at the Omni Nashville Hotel', u'latitude': 36.1658333} ****************************************************
1
Jarett McAlister {u'city': u'Nashville', u'name': u'Barlines at the Omni Nashville Hotel', u'country': u'United States', u'region': u'TN', u'longitude': -86.7844444, u'place': u'Barlines at the Omni Nashville Hotel', u'latitude': 36.1658333} ****************************************************
Jarett McAlister {u'city': u'Nashville', u'name': u'Barlines at the Omni Nashville Hotel', u'country': u'United States', u'region': u'TN', u'longitude': -86.7844444, u'place': u'Barlines at the Omni Nashville Hotel', u'latitude': 36.1658333} ****************************************************
Jarett McAlister {u'city': u'Nashville', u'name': u'Barlines at the Omni Nashville Hotel', u'country': u'United Sta

MERGE Cuisine & Wine Bistro WITH Cuisine and Wine Bistro ? [y/N] y
True
Cameron DeGurski {u'city': u'Gilbert', u'name': u'Cuisine & Wine Bistro', u'country': u'United States', u'region': u'AZ', u'longitude': -111.8212783, u'place': u'Cuisine & Wine Bistro', u'latitude': 33.3359143} ****************************************************
Cameron DeGurski {u'city': u'Gilbert', u'name': u'Cuisine & Wine Bistro', u'country': u'United States', u'region': u'AZ', u'longitude': -111.8212783, u'place': u'Cuisine & Wine Bistro', u'latitude': 33.3359143} ****************************************************
2
The Upper Strata {u'city': u'Gilbert', u'name': u'Cuisine & Wine Bistro', u'country': u'United States', u'region': u'AZ', u'longitude': -111.8212783, u'place': u'Cuisine & Wine Bistro', u'latitude': 33.3359143} ****************************************************
The Upper Strata {u'city': u'Gilbert', u'name': u'Cuisine & Wine Bistro', u'country': u'United States', u'region': u'AZ', u'longitude'

_______________________________
AAAA  => Fins Bar & Grille 
appxAAA ltlng=> 38.935 -74.906
AAAA gigs => 14
BBBB  => Dillian's Bar & Grill
appxBBB ltlng=> 38.893 -121.293
BBBB gigs => 9
MERGE Dillian's Bar & Grill WITH Fins Bar & Grille  ? [y/N] y
True
Big Trouble {u'city': u'Lincoln', u'name': u"Dillian's Bar & Grill", u'country': u'United States', u'region': u'CA', u'longitude': -121.292811, u'place': u"Dillian's Bar & Grill", u'latitude': 38.893315} ****************************************************
Big Trouble {u'city': u'Lincoln', u'name': u"Dillian's Bar & Grill", u'country': u'United States', u'region': u'CA', u'longitude': -121.292811, u'place': u"Dillian's Bar & Grill", u'latitude': 38.893315} ****************************************************
Big Trouble {u'city': u'Lincoln', u'name': u"Dillian's Bar & Grill", u'country': u'United States', u'region': u'CA', u'longitude': -121.292811, u'place': u"Dillian's Bar & Grill", u'latitude': 38.893315} ******************************

KeyboardInterrupt: 

In [ ]:
##CHECK FOR DATE DUPLICATES
for artist in artists.find():
    seen = set()
    uniq = []
    counter=0
    for gig in artist["gigs"]:
         counter +=1
         if gig["datetime"] not in seen:
            uniq.append(x)
            seen.add(x)
    if   counter != len(uniq) :
        print "DUP"
        break
        
        
    